In [ ]:
import pandas as pd
from collections import defaultdict
import pickle

In [ ]:
meal_statistics = pd.read_csv('data/mealstatistics.csv')
meals = pd.read_csv('data/meals.csv')
meal_history = pd.read_csv('data/mealhistory_latest.csv')
canteens = pd.read_csv('data/canteens.csv')

In [ ]:
full_meal_history = pd.read_csv('data/mealhistory.csv')

In [ ]:
meals_db = dict()
for i, row in canteens.iterrows():
    meals_db[row['id']] = defaultdict(dict)


In [ ]:
i = 0
for _, row in full_meal_history.iterrows():
    kind_id = row['kindId']
    if kind_id == 3:
        to_append = (row['timestamp'], row['countAvailable'], row['countOrdered'])
        if row['altId'] not in meals_db[row['canteenId']][row['day']].keys():
            meals_db[row['canteenId']][row['day']][row['altId']] = []
        meals_db[row['canteenId']][row['day']][row['altId']].append(to_append)
    i += 1
    if i % 10000 == 0:
        print(i)


In [ ]:
with open('data/meals_db', 'wb') as f:
    pickle.dump(meals_db, f)


In [ ]:
with open('data/meals_db', 'rb') as f:
    meals_db = pickle.load(f)


In [ ]:
for k, v in meals_db.items():
    for day, meal in v.items():
        for it, l in meal.items():
            meal[it] = sorted(l, key=lambda x: x[0])

In [ ]:
with open('data/meals_db_2', 'wb') as f:
    pickle.dump(meals_db, f)

In [ ]:
with open('data/meals_db_2', 'rb') as f:
    meals_db = pickle.load(f)

In [ ]:
def is_during_opening_hours(tupl):
    tmp = tupl[0].split(' ')[1]
    if "10:30:00" < tmp < "14:31:00" and tupl[0] > "2022-09-30":
        return True
    return False

for k, v in meals_db.items():
    for day, meal in v.items():
        for it, l in meal.items():
            meal[it] = list(filter(is_during_opening_hours, l))
    print('k', end='')


In [ ]:
with open('data/meals_db_3', 'wb') as f:
    pickle.dump(meals_db, f)

In [3]:
import pandas as pd
from collections import defaultdict
import pickle

In [4]:
with open('data/meals_db_3', 'rb') as f:
    meals_db = pickle.load(f)

In [5]:
def check_if_no_dup(l):
    if len(l) == 0:
        return
    for i in range(len(l) - 1):
        if l[i][0][:20] == l[i + 1][0][:20]:
            print(l[i], l[i + 1])

In [6]:
# deduplicates and also removes miliseconds and time zone info
def deduplicate(l):
    if len(l) == 0:
        return []

    res = []

    for i in range(len(l) - 1):
        if l[i][0][:16] == l[i+1][0][:16]:
            if l[i][1] != l[i+1][1] or l[i][2] != l[i+1][2]:
                pass
                #print(l[i], l[i+1])
            continue
        else:
            res.append((l[i][0][:16], l[i][1], l[i][2]))
    res.append((l[-1][0][:16], l[-1][1], l[-1][2]))
    return res

for k, v in meals_db.items():
    for day, meal in v.items():
        for it, l in meal.items():
            meal[it] = deduplicate(l)
    print('k', end='')

kkkkkkkkkkkkkkkkkkkkkkkkk

In [7]:
for k, v in meals_db.items():
    for day, meal in v.items():
        for it, l in meal.items():
            check_if_no_dup(l)
    print('k', end='')

kkkkkkkkkkkkkkkkkkkkkkkkk

In [8]:
def check_ordered(l):
    if len(l) == 0:
        return
    for i in range(len(l) - 1):
        if l[i][2] > l[i + 1][2]:
            print(l[i], l[i + 1])

for k, v in meals_db.items():
    for day, meal in v.items():
        for it, l in meal.items():
            check_ordered(l)
    print('k', end='')

k('2022-10-03 11:43', -1, 15) ('2022-10-03 11:44', -1, 13)
('2022-10-04 13:12', -1, 22) ('2022-10-04 13:13', -1, 21)
('2022-10-04 11:32', -1, 4) ('2022-10-04 11:33', -1, 1)
('2022-10-05 12:37', -1, 35) ('2022-10-05 12:38', -1, 34)
('2022-10-05 13:58', -1, 41) ('2022-10-05 13:59', -1, 40)
('2022-10-10 11:35', -1, 9) ('2022-10-10 11:36', -1, 7)
('2022-10-14 11:24', -1, 2) ('2022-10-14 11:25', -1, 1)
('2022-10-14 11:37', -1, 4) ('2022-10-14 11:38', -1, 3)
('2022-10-17 11:41', -1, 5) ('2022-10-17 11:42', -1, 4)
('2022-10-17 11:41', -1, 7) ('2022-10-17 11:42', -1, 6)
('2022-10-19 13:09', -1, 25) ('2022-10-19 13:10', -1, 24)
('2022-10-20 11:33', -1, 8) ('2022-10-20 11:34', -1, 7)
('2022-10-20 12:16', -1, 15) ('2022-10-20 12:17', -1, 14)
('2022-10-20 12:01', -1, 22) ('2022-10-20 12:02', -1, 21)
('2022-10-20 11:38', -1, 17) ('2022-10-20 11:39', -1, 16)
('2022-10-24 11:27', -1, 8) ('2022-10-24 11:28', -1, 7)
('2022-10-24 11:46', -1, 16) ('2022-10-24 11:47', -1, 15)
('2022-10-24 11:32', -1, 5) (

In [9]:
# sometimes, there are some undos done and then the amount of ordered decreases and amount of available increases. Try to revert it by decreasing previous amount until there is no negative increase of ordered.
def revert_undos(l):
    if len(l) == 0:
        return []

    i = len(l) - 1
    while i > 0:
        if l[i][2] < l[i - 1][2]:
            diff = l[i - 1][2] - l[i][2]
            if l[i-1][1] == l[i][1]:
                print(l[i - 1], '->', (l[i-1][0], l[i-1][1], l[i-1][2] - diff), 'then', l[i])
                l[i-1] = (l[i-1][0], l[i-1][1], l[i-1][2] - diff)
            else:
                print(l[i - 1], '->', (l[i-1][0], l[i-1][1] + diff, l[i-1][2] - diff), 'then', l[i])
                l[i-1] = (l[i-1][0], l[i-1][1] + diff, l[i-1][2] - diff)
        i -= 1

for k, v in meals_db.items():
    for day, meal in v.items():
        for it, l in meal.items():
            revert_undos(l)
    print('k', end='')

k('2022-10-03 11:43', -1, 15) -> ('2022-10-03 11:43', -1, 13) then ('2022-10-03 11:44', -1, 13)
('2022-10-04 13:12', -1, 22) -> ('2022-10-04 13:12', -1, 21) then ('2022-10-04 13:13', -1, 21)
('2022-10-04 11:32', -1, 4) -> ('2022-10-04 11:32', -1, 1) then ('2022-10-04 11:33', -1, 1)
('2022-10-05 13:58', -1, 41) -> ('2022-10-05 13:58', -1, 40) then ('2022-10-05 13:59', -1, 40)
('2022-10-05 12:37', -1, 35) -> ('2022-10-05 12:37', -1, 34) then ('2022-10-05 12:38', -1, 34)
('2022-10-10 11:35', -1, 9) -> ('2022-10-10 11:35', -1, 7) then ('2022-10-10 11:36', -1, 7)
('2022-10-14 11:24', -1, 2) -> ('2022-10-14 11:24', -1, 1) then ('2022-10-14 11:25', -1, 1)
('2022-10-14 11:37', -1, 4) -> ('2022-10-14 11:37', -1, 3) then ('2022-10-14 11:38', -1, 3)
('2022-10-17 11:41', -1, 5) -> ('2022-10-17 11:41', -1, 4) then ('2022-10-17 11:42', -1, 4)
('2022-10-17 11:41', -1, 7) -> ('2022-10-17 11:41', -1, 6) then ('2022-10-17 11:42', -1, 6)
('2022-10-19 13:09', -1, 25) -> ('2022-10-19 13:09', -1, 24) then (

In [10]:
for k, v in meals_db.items():
    for day, meal in v.items():
        for it, l in meal.items():
            check_ordered(l)
    print('k', end='')

kkkkkkkkkkkkkkkkkkkkkkkkk

In [11]:
def fast_compute_of_serverd_meals(canteen):
    total_served = 0
    for day, meal in canteen.items():
        for it, l in meal.items():
            if len(l) > 0:
                total_served += l[-1][2]
    return total_served

keys_to_delete = []
for k, v in meals_db.items():
    served = fast_compute_of_serverd_meals(v)
    print('canteen', k, 'server', served, 'meals')
    if served < 15000:
        keys_to_delete.append(k)

keys_to_delete.append(10)
for k in keys_to_delete:
    del meals_db[k]

print(meals_db.keys())
for k, v in meals_db.items():
    served = fast_compute_of_serverd_meals(v)
    print('canteen', k, 'server', served, 'meals')


canteen 55 server 1 meals
canteen 12 server 4888 meals
canteen 10 server 36392 meals
canteen 11 server 2307 meals
canteen 1 server 0 meals
canteen 5 server 69522 meals
canteen 59 server 11151 meals
canteen 7 server 79686 meals
canteen 8 server 12157 meals
canteen 15 server 26154 meals
canteen 18 server 0 meals
canteen 60 server 4021 meals
canteen 21 server 21950 meals
canteen 54 server 15422 meals
canteen 61 server 7970 meals
canteen 29 server 17 meals
canteen 27 server 78999 meals
canteen 30 server 0 meals
canteen 38 server 8286 meals
canteen 35 server 51616 meals
canteen 37 server 0 meals
canteen 52 server 0 meals
canteen 42 server 0 meals
canteen 50 server 0 meals
canteen 40 server 0 meals
dict_keys([5, 7, 15, 21, 54, 27, 35])
canteen 5 server 69522 meals
canteen 7 server 79686 meals
canteen 15 server 26154 meals
canteen 21 server 21950 meals
canteen 54 server 15422 meals
canteen 27 server 78999 meals
canteen 35 server 51616 meals


In [12]:
# interpolate
from datetime import datetime, timedelta
import numpy as np

def interpolate(l):
    if len(l) == 0:
        return []

    with_datetimes = []
    for i in range(len(l)):
        tmp = datetime.strptime(l[i][0], "%Y-%m-%d %H:%M")
        with_datetimes.append((tmp, l[i][1], l[i][2]))

    if with_datetimes[0][0].hour != 10 or with_datetimes[0][0].minute != 30:
        if with_datetimes[0][2] > 0:
            po = 6
        with_datetimes.insert(0, (with_datetimes[0][0] - timedelta(minutes=1), 0, 0))
        if with_datetimes[0][0].hour != 10 or with_datetimes[0][0].minute != 30:
            with_datetimes.insert(0, (datetime.strptime(l[0][0][:11] + "10:30", "%Y-%m-%d %H:%M"), 0, 0))
        po = 7
    if with_datetimes[-1][0].hour != 14 or with_datetimes[-1][0].minute != 30:
        # add value 1 minute later when there are no free meals
        with_datetimes.append((with_datetimes[-1][0] + timedelta(minutes=1), 0, with_datetimes[-1][2]))
        if with_datetimes[-1][0].hour != 14 or with_datetimes[-1][0].minute != 30:
            # add value at the end of the day, the same as the last one
            with_datetimes.append((datetime.strptime(l[0][0][:11] + "14:30", "%Y-%m-%d %H:%M"), 0, with_datetimes[-1][2]))
        po = 8

    res = []
    for i in range(len(with_datetimes) - 1):
        res.append(with_datetimes[i])

        time_diff_minutes = int((with_datetimes[i + 1][0] - with_datetimes[i][0]).total_seconds() / 60)
        if time_diff_minutes != 1:
            # interpolate
            start = with_datetimes[i][2]
            end = with_datetimes[i + 1][2]
            interpolated_values = np.linspace(start, end, time_diff_minutes + 1)

            available_start = with_datetimes[i][1]
            available_end = with_datetimes[i + 1][1]
            available_interpolated_values = np.linspace(available_start, available_end, time_diff_minutes + 1)

            for j in range(1, time_diff_minutes):
                res.append((with_datetimes[i][0] + timedelta(minutes=j), int(round(available_interpolated_values[j])), int(round(interpolated_values[j]))))

    res.append(with_datetimes[-1])
    return res

for k, v in meals_db.items():
    for day, meal in v.items():
        for it, l in meal.items():
            meal[it] = interpolate(l)
    print('k', end='')


kkkkkkk

In [13]:
# remove days in the September
for k, v in meals_db.items():
    to_delete = []
    for day, meal in v.items():
        if day < '2022-10-01':
            to_delete.append(day)
    for day in to_delete:
        del v[day]


In [14]:
# add 2 specific days for canteens so we have data for all
cant1 = meals_db[27]
cant1['2022-10-11'] = dict()
cant1['2022-10-11'][1] = []
cant1['2022-10-11'][2] = []
cant1['2022-10-11'][3] = []
cant1['2022-10-11'][4] = []

meals_db[27] = dict(sorted(cant1.items(), key=lambda item: item[0]))

cant2 = meals_db[35]
cant2['2022-10-27'] = dict()
cant2['2022-10-27'][1] = []
cant2['2022-10-27'][2] = []
cant2['2022-10-27'][3] = []
cant2['2022-10-27'][4] = []

meals_db[35] = dict(sorted(cant2.items(), key=lambda item: item[0]))


In [15]:
x = 7

In [16]:
# remove days where there were all canteens closed
all_possible_days = list(meals_db[27].keys())

to_delete = []
for target_day in all_possible_days:
    summed = 0
    for k, v in meals_db.items():
        for day, meal in v.items():
            if day == target_day:
                for m in meal.values():
                    summed += len(m)
    if summed == 0:
        to_delete.append(target_day)

to_delete.append('2023-04-25')

for k, v in meals_db.items():
    for day in to_delete:
        if day in v:
            del v[day]

In [17]:
# remove meals with no records
for canteen_id, day_dicts in meals_db.items():
    for day, meal_lists in day_dicts.items():
        to_delete = []
        for meal_id, l in meal_lists.items():
            if len(l) == 0:
                to_delete.append(meal_id)
        for meal_id in to_delete:
            del meal_lists[meal_id]

In [18]:
with open('data/meals_db_4', 'wb') as f:
    pickle.dump(meals_db, f)

In [ ]:
x = 7